In [2]:
import pandas as pd
import os
dataset_folder = r"D:\Code\Project\Amazon\data"

# Load raw data
amazon_data = pd.read_csv(os.path.join(dataset_folder, "amazon_india_complete_2015_2025.csv"))
amazon_data_original = amazon_data.copy()



In [3]:
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime


class AmazonDataCleaner:
    def __init__(self, dataset_folder):
        self.dataset_folder = dataset_folder
        self.amazon_data = None

    # Load Data
    def load_data(self):
        try:
            self.amazon_data = pd.read_csv(
                os.path.join(self.dataset_folder, "amazon_india_complete_2015_2025.csv")
            )
            print(f"Amazon Complete loaded: {self.amazon_data.shape}")

        except FileNotFoundError as e:
            print(f"Error: {e}")
            print("Please check dataset path.")

# Clean Columns
    def clean_columns(self):
        # order_date
        self.amazon_data['order_date'] = pd.to_datetime(
            self.amazon_data['order_date'], 
            errors='coerce',     # turn bad ones into NaT
            dayfirst=True,       # needed for DD-MM-YYYY and DD/MM/YYYY
            format="mixed"       # <-- NEW in Pandas 2.0, handles mixed styles
        )
        self.amazon_data['original_price_inr'] = (
            self.amazon_data['original_price_inr'].str.replace("Rs", "", regex=False)
    .str.replace("â‚¹", "", regex=False)   
    .str.replace("₹", "", regex=False)    
    .str.replace(",", "", regex=False)
    .str.replace('-', '', regex=False)
    .str.strip()
    .astype(float)
    .round(2))
    # customer_rating
        self.amazon_data['customer_rating'] = (
            self.amazon_data['customer_rating'].astype(str)
    .str.lower()
    .str.strip()
    .str.replace('stars', '', regex=False)
    .str.replace('star', '', regex=False)

)
        self.amazon_data['category'] = (
            self.amazon_data['category']
            .str.strip().str.lower()
            .replace({'Electronics': 'Electronics',
    'Electronic': 'Electronics',
    'ELECTRONICS': 'Electronics',
    'Electronics & Accessories': 'Electronics',
    'Electronicss': 'Electronics',
}   )  )
# City Cleaning
    def clean_cities(self):
        self.amazon_data['customer_city'] = (
            self.amazon_data['customer_city'].str.strip().str.lower()
        )
        city_mapping = {"kolkata": "Kolkata", "calcutta": "Kolkata",

    "bangalore": "Bengaluru", "bengaluru": "Bengaluru",
    "banglore": "Bengaluru", "bengalore": "Bengaluru",
    "bangalore/bengaluru": "Bengaluru",

    "mumbai": "Mumbai", "bombay": "Mumbai", "mumba": "Mumbai",

    "chennai": "Chennai", "chenai": "Chennai", "madras": "Chennai",

    "delhi": "Delhi", "new delhi": "Delhi", "delhi ncr": "Delhi",

    "kochi": "Kochi", "ludhiana": "Ludhiana", "aligarh": "Aligarh",
    "surat": "Surat", "kanpur": "Kanpur", "hyderabad": "Hyderabad",
    "bareilly": "Bareilly", "vadodara": "Vadodara", "indore": "Indore",
    "visakhapatnam": "Visakhapatnam", "lucknow": "Lucknow", "pune": "Pune",
    "bhubaneswar": "Bhubaneswar", "nagpur": "Nagpur", "patna": "Patna",
    "ahmedabad": "Ahmedabad", "jaipur": "Jaipur", "meerut": "Meerut",
    "varanasi": "Varanasi", "coimbatore": "Coimbatore",
    "moradabad": "Moradabad", "saharanpur": "Saharanpur",
    "chandigarh": "Chandigarh", "gorakhpur": "Gorakhpur",
    "allahabad": "Prayagraj",
}

        self.amazon_data['customer_city'] = (
            self.amazon_data['customer_city']
            .map(city_mapping)
            .fillna(self.amazon_data['customer_city'])
            .str.title()
        )
    def normalize_booleans(self):
      bool_map = {
        'true': True, 'yes': True, 'y': True, '1': True,
        'false': False, 'no': False, 'n': False, '0': False
      }

      for col in ['is_festival_sale', 'is_prime_member', 'is_prime_eligible']:
        s = (
        self.amazon_data[col]
            .astype("string")
            .str.strip()
            .str.lower()
        )

        self.amazon_data[col] = s.map(bool_map).astype("boolean")

# Standardize Customer Rating
    def clean_customer_rating(self):
      def parse_rating(r):
        if pd.isna(r):
            return np.nan

        r = str(r).strip().lower()

        # Handle fraction ratings like 4/5
        if '/' in r:
            try:
                num, den = r.split('/')
                return (float(num) / float(den)) * 5
            except (ValueError, ZeroDivisionError):
                return np.nan

        # Handle numeric ratings like "4.5 stars"
        match = re.search(r'(\d+(\.\d+)?)', r)
        if match:
            rating = float(match.group(1))
            return rating if 0 <= rating <= 5 else np.nan

        return np.nan

      self.amazon_data['customer_rating'] = (
        self.amazon_data['customer_rating']
        .apply(parse_rating)
    )
    # Fill missing ratings by customer_tier mean
      self.amazon_data['customer_rating'] = self.amazon_data.groupby('customer_tier')['customer_rating'].transform(
        lambda x: x.fillna(x.mean())
    )

    # Fill any remaining NaN with overall mean
      self.amazon_data['customer_rating'] = self.amazon_data['customer_rating'].fillna(
        self.amazon_data['customer_rating'].mean()
    )

    # Round final ratings to 1 decimal
      self.amazon_data['customer_rating'] = self.amazon_data['customer_rating'].round(1)
    def clean_payment_method(self):
        mapping ={
    # UPI
    'upi': 'UPI',
    'phonepe': 'UPI',
    'googlepay': 'UPI',
    'gpay': 'UPI',

    # Credit Card
    'credit card': 'Credit Card',
    'cc': 'Credit Card',
    'credit_card': 'Credit Card',

    # Debit Card
    'debit card': 'Debit Card',
    'dc': 'Debit Card',
    # COD
    'cash on delivery': 'COD',
    'cod': 'COD',
    'c.o.d': 'COD',

    # Net Banking
    'net banking': 'Net Banking',
    'netbanking': 'Net Banking',

    # Wallet
    'wallet': 'Wallet',

    # Buy Now Pay Later
    'bnpl': 'BNPL'
}
        
        self.amazon_data['payment_method'] = (
            self.amazon_data['payment_method']
            .astype(str).str.strip().str.lower()
            .map(mapping)
            .fillna(self.amazon_data['payment_method'].str.title())
        )
        # Handle Missing Values
    def handle_missing_values(self):
        # Fill missing delivery_charges with median (to reduce effect of outliers)
        self.amazon_data['delivery_charges'] = self.amazon_data['delivery_charges'].fillna(
            self.amazon_data['delivery_charges'].median()
        )
        
        # Fill missing customer_age_group with 'Unknown'
        self.amazon_data['customer_age_group'] = self.amazon_data['customer_age_group'].fillna('Unknown')
        
        # Fill missing festival_name based on 'is_festival_sale'
        self.amazon_data.loc[
            self.amazon_data['is_festival_sale'] == False, 
            'festival_name'
        ] = 'None'
        self.amazon_data['festival_name'] = self.amazon_data['festival_name'].fillna('Other')
        
        # Fill missing customer_rating with average rating per tier (or overall mean)
        self.amazon_data['customer_rating'] = self.amazon_data.groupby('customer_tier')['customer_rating'].transform(
            lambda x: x.fillna(x.mean())
        )
        # If still some nulls left, fill with overall mean
        self.amazon_data['customer_rating'] = self.amazon_data['customer_rating'].fillna(
            self.amazon_data['customer_rating'].mean()
        )
        
        print("Missing values handled!")


    # Handle Duplicates with Advanced Classification
    def handle_duplicates(self, agg_bulk=False):
        duplicate_cols = [
            'order_date', 'customer_id', 'product_id',
            'original_price_inr', 'discounted_price_inr', 'final_amount_inr'
        ]
        quantity_col = 'quantity'
        final_amount_col = 'final_amount_inr'
        
        # Mark all duplicates
        self.amazon_data['is_duplicate'] = self.amazon_data.duplicated(subset=duplicate_cols, keep=False)
        
        # Group duplicates
        grouped = self.amazon_data[self.amazon_data['is_duplicate']].groupby(duplicate_cols)
        
        # Classify each duplicate group
        def classify_duplicate(group):
            total_quantity = group[quantity_col].sum()
            total_amount = group[final_amount_col].sum()
            price_per_unit = group[final_amount_col].iloc[0] / group[quantity_col].iloc[0]
            expected_total = total_quantity * price_per_unit
            if abs(expected_total - total_amount) < 1e-2:
                return "bulk_order"
            else:
                return "error_duplicate"
        
        duplicate_classification = (
            grouped.apply(classify_duplicate)
            .reset_index(name='duplicate_type')
        )
        
        # Merge back
        self.amazon_data = self.amazon_data.merge(duplicate_classification, on=duplicate_cols, how='left')
        
        # Aggregate bulk orders if needed
        if agg_bulk:
            bulk_orders = self.amazon_data[self.amazon_data['duplicate_type'] == "bulk_order"]
            aggregated = bulk_orders.groupby(duplicate_cols, as_index=False).agg({
                quantity_col: 'sum',
                'subtotal_inr': 'sum',
                final_amount_col: 'sum',
                'transaction_id': lambda x: ','.join(x)
            })
            non_bulk = self.amazon_data[self.amazon_data['duplicate_type'] != "bulk_order"]
            self.amazon_data = pd.concat([non_bulk, aggregated], ignore_index=True)
        
        return self.amazon_data
        # Outlier Detection - IQR Method
    def flag_price_outliers_iqr(self):
        df = self.amazon_data

        q1 = df.groupby(['subcategory', 'brand'])['original_price_inr'].transform('quantile', 0.25)
        q3 = df.groupby(['subcategory', 'brand'])['original_price_inr'].transform('quantile', 0.75)

        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr

        df['price_outlier_IQR'] = (
            (df['original_price_inr'] < lower) |
            (df['original_price_inr'] > upper)
    )
        # Flag All Outliers 
    def flag_outliers(self):
        self.flag_price_outliers_iqr()


    # Run All Steps
    def clean_all(self):
        self.load_data()
        self.clean_columns()
        self.clean_cities()
        self.normalize_booleans()
        self.clean_customer_rating()
        self.clean_payment_method()
        self.handle_missing_values()
        self.handle_duplicates()
        self.flag_outliers()
        print("Data Cleaning Completed!")


    



In [6]:
dataset_folder = r"D:\Code\Project\Amazon\data" 
cleaner = AmazonDataCleaner(dataset_folder) 
cleaner.clean_all() 
amazon_cleaned_data = cleaner.amazon_data 
amazon_cleaned_data.info()

Amazon Complete loaded: (1127609, 34)
Missing values handled!


C:\Users\Dell\AppData\Local\Temp\ipykernel_47016\2259502010.py:246: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(classify_duplicate)


Data Cleaning Completed!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127609 entries, 0 to 1127608
Data columns (total 37 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   transaction_id          1127609 non-null  object        
 1   order_date              1127609 non-null  datetime64[ns]
 2   customer_id             1127609 non-null  object        
 3   product_id              1127609 non-null  object        
 4   product_name            1127609 non-null  object        
 5   category                1127609 non-null  object        
 6   subcategory             1127609 non-null  object        
 7   brand                   1127609 non-null  object        
 8   original_price_inr      1127609 non-null  float64       
 9   discount_percent        1127609 non-null  float64       
 10  discounted_price_inr    1127609 non-null  float64       
 11  quantity                1127609 non-null  int64    

In [54]:
output_path = os.path.join(dataset_folder, "amazon_india_cleaned_data.csv")
amazon_cleaned_data.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: D:\Code\Project\Amazon\data\amazon_india_cleaned_data.csv


In [57]:
numeric_cols = amazon_data_original.select_dtypes(include=np.number).columns

summary_comparison = pd.concat(
    [
        amazon_data_original[numeric_cols].describe().T.add_prefix("Original_"),
        amazon_cleaned_data[numeric_cols].describe().T.add_prefix("Cleaned_")
    ],
    axis=1
)

summary_comparison


,Original_count,Original_mean,Original_std,Original_min,Original_25%,Original_50%,Original_75%,Original_max,Cleaned_count,Cleaned_mean,Cleaned_std,Cleaned_min,Cleaned_25%,Cleaned_50%,Cleaned_75%,Cleaned_max
discount_percent,1127609.0,17.420350,20.553770,0.00,0.00,10.37,28.41,70.00,1127609.0,17.420350,20.553770,0.00,0.00,10.37,28.41,70.00
discounted_price_inr,1127609.0,54541.339570,45824.804426,344.33,22780.69,38001.19,74103.32,420704.77,1127609.0,54541.339570,45824.804426,344.33,22780.69,38001.19,74103.32,420704.77
quantity,1127609.0,1.250063,0.536503,1.00,1.00,1.00,1.00,3.00,1127609.0,1.250063,0.536503,1.00,1.00,1.00,1.00,3.00
subtotal_inr,1127609.0,68187.402906,68934.068999,344.33,25217.99,44731.82,88521.79,1262114.31,1127609.0,68187.402906,68934.068999,344.33,25217.99,44731.82,88521.79,1262114.31
delivery_charges,1037408.0,0.000308,0.111078,0.00,0.00,0.00,0.00,40.00,1127609.0,0.000284,0.106543,0.00,0.00,0.00,0.00,40.00
final_amount_inr,1127609.0,68187.403225,68934.068685,344.33,25217.99,44731.82,88521.79,1262114.31,1127609.0,68187.403225,68934.068685,344.33,25217.99,44731.82,88521.79,1262114.31
order_month,1127609.0,6.940627,3.539183,1.00,4.00,7.00,10.00,12.00,1127609.0,6.940627,3.539183,1.00,4.00,7.00,10.00,12.00
order_year,1127609.0,2020.637254,2.690906,2015.00,2019.00,2021.00,2023.00,2025.00,1127609.0,2020.637254,2.690906,2015.00,2019.00,2021.00,2023.00,2025.00
order_quarter,1127609.0,2.645922,1.140795,1.00,2.00,3.00,4.00,4.00,1127609.0,2.645922,1.140795,1.00,2.00,3.00,4.00,4.00
product_weight_kg,1127609.0,0.780698,3.674216,0.03,0.18,0.21,0.24,44.26,1127609.0,0.780698,3.674216,0.03,0.18,0.21,0.24,44.26
